<a href="https://colab.research.google.com/github/rafabrnd/Coursera_Capstone/blob/master/coursera_capstone_Kmeans_torrontoneighborhoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Segmenting and Clustering Neighborhoods in Toronto**
### *Applied Data Science Capstone > Week 3*






<img src="https://media.giphy.com/media/olAik8MhYOB9K/giphy.gif" width="250" align="left">



Load the necessary packages

In [0]:
import numpy as np
import pandas as pd


### 1. Load the file URL from GitHub
Ignore some bad lines where there was some formating errors. To get to this dataset I copied the table into MS Excel and saved the file as .csv (comma separated)  

In [363]:
url = 'https://raw.githubusercontent.com/rafabrnd/Coursera_Capstone/master/toronto_neigh_dataset.csv' 
toronto = pd.read_csv(url,sep=',',error_bad_lines=False) #here i ignored some bad lines to make it possible to load the file
toronto.head() #check the first 5 rows of the dataset

b'Skipping line 15: expected 3 fields, saw 4\n'


,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [364]:
#check the shape of the dataset

print("Dataset shape is: ", toronto.shape)
print ('Columns names are: ', toronto.columns)


Dataset shape is:  (182, 3)
Columns names are:  Index(['Postal code ', 'Borough ', 'Neighborhood'], dtype='object')


### 2. Data wrangling

Here I noticed that the 'Postal code ' and 'Borough ' have a space after their name. 
Their names should be replaced for something easier to read


In [365]:
toronto = toronto.rename(columns={'Postal code ':'postal_code','Borough ':'borough', 'Neighborhood':'neighborhood'})
toronto.columns

Index(['postal_code', 'borough', 'neighborhood'], dtype='object')

Here all the entries where the borough is 'Not assinged ' are droped.
Notice that there is a ' ' at the right side.
A new dataframe is defined with all the rows are that have assigned borough names

In [0]:
toronto = toronto.loc[~(toronto['borough'] == 'Not assigned ')]

In [367]:
print("The current shape of the Toronto datafrane is: ", toronto.shape)
toronto.head()

The current shape of the Toronto datafrane is:  (105, 3)


,postal_code,borough,neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


Remove all rows that contain 'NaN' values.
Due to the copy and paste & the formating on wikipedia page some rows show NaN values even though they are not suposed to be a number anyway.
Index was reset (drop=True)

In [368]:
#
toronto = toronto.dropna().reset_index(drop=True)
toronto.head()

,postal_code,borough,neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


Due to the copy & paste from the wikipedia page into the MS Excel there were some residual spaces either before or after some of the values. Here I used a **for** loop that loops through the different column names and strips each Series from potential spaces (' ') that could be located at their left or right side.
Note that this strip() function does not remove the spaces in the middle of the strings, so it is safe to use even though we have some 2 work names. 

In [369]:
columns = list(toronto.columns)
for column_name in columns:
  toronto[column_name] = toronto[column_name].str.strip()
toronto.head()

,postal_code,borough,neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [370]:
#replace the divider ' / ' with ','

toronto.loc[toronto['neighborhood'] == 'Regent Park / Harbourfront']
toronto['neighborhood'] = toronto['neighborhood'].replace(' / ',', ', regex=True)
toronto.head()

,postal_code,borough,neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"



##<font color=red>This might not be necessary; Delete if not needed</font>

Here I split the neighborhood names at the '/' in order to create a list with each one of those names

The new list (neighborhood_list) was then added to the original dataframe (toronto)

I decided to create a list in each of the neighborhood values because it might be easier to access individual names of neighnorhoods if necessay.

In [371]:
#split toronto on their '/' and it becomes a Series with a list on each value
neighborhood_list = toronto['neighborhood'].str.split('/', n=-1)
neighborhood_list = neighborhood_list.reset_index(drop=True) #reset the index

#add to the original datafraome
toronto_list = toronto
toronto_list['neighborhood'] = neighborhood_list
toronto_list.head()



,postal_code,borough,neighborhood
0,M3A,North York,[Parkwoods]
1,M4A,North York,[Victoria Village]
2,M5A,Downtown Toronto,"[Regent Park, Harbourfront]"
3,M6A,North York,"[Lawrence Manor, Lawrence Heights]"
4,M7A,Downtown Toronto,"[Queen's Park, Ontario Provincial Government]"
